在crewai中，Agent相当于团队中的一个成员，不同代理具有不同的角色、擅长领域及内在知识，Agent有以下作用：

1. 绑定到任务中，让其执行特定任务
2. 根据角色和目标作出决策
3. 使用工具
4. 与其他代理进行沟通
5. 委派任务给其他代理

总之一句话，Agent有自己的知识及解决工具，当自己能力不足时，可以请求团队内其他人协助

关键参数：

|属性|参数|类型|描述|
|---|---|---|---|
|角色|role|str|定义代理在团队中的职能和专业知识。|
|目标|goal|str|指导代理决策的单个目标。|
|背景故事|backstory|str|为代理提供上下文和个性，丰富交互。|
|LLM（可选）|llm|Union[str, LLM, Any]|为代理提供支持的语言模型。默认为 “gpt-4” 中指定的模型。OPENAI_MODEL_NAME|
|工具（可选）|tools|List[BaseTool]|代理可用的功能。默认为空列表。|
|函数调用 LLM（可选）|function_calling_llm|Optional[Any]|用于工具调用的语言模型，如果指定，则覆盖班组的 LLM。|
|内存（可选）|memory|bool|代理是否应维护交互的记忆。默认值为 True。|
|详细 （可选）|verbose|bool|启用详细的执行日志以进行调试。默认值为 False。|
|允许委派（可选）|allow_delegation|bool|允许代理将任务委派给其他代理。默认值为 False。|
|缓存 （可选）|cache|bool|为工具使用启用缓存。默认值为 True。|
|系统模板（可选）|system_template|Optional[str]|代理的自定义系统提示模板。|
|提示模板 （可选）|prompt_template|Optional[str]|代理的自定义提示模板。|
|响应模板（可选）|response_template|Optional[str]|代理的自定义响应模板。|
|允许代码执行（可选）|allow_code_execution|Optional[bool]|为代理启用代码执行。默认值为 False。|
|Embedder Config（可选）|embedder_config|Optional[Dict[str, Any]]|代理使用的嵌入器的配置。|

In [32]:
# 定义Agent
from api_key import SERPER_API_KEY,TAVILY_API_KEY
import os
os.environ["SERPER_API_KEY"] = SERPER_API_KEY  # serper.dev API key



from crewai import  Agent
from crewai_tools import SerperDevTool
from crewai.llm import LLM


cv_agent=Agent(
    role="计算机视觉工程师",
    goal="查找并理解计算机视觉工程师的工作内容，熟悉的代码和工具",
    backstory = "已经在计算机视觉领域工作5年，有丰富的计算机视觉经验，尤其是在工业缺陷检测方面",
    llm=LLM(
        model="ollama_chat/llama3.1:latest", 
        base_url="http://192.168.3.155:11434"),
    memory=True,
    verbose=True,
    tools=[SerperDevTool(n_results=3)],
    embedder_config={
        "provider": "ollama",
        "config": {
            "model": 'quentinz/bge-large-zh-v1.5:latest',
            "base_url": 'http://localhost:11434'
        }
    }
)

nlp_agent=Agent(
    role="自然语言开发工程师",
    goal="查找并理解计算机视觉工程师的工作内容，熟悉的代码和工具",
    backstory = "已经在自然语言开发领域工作10年，有丰富的自然语言开发经验",
    llm=LLM(
        model="ollama_chat/llama3.1:latest", 
        base_url="http://192.168.3.155:11434"),
    memory=True,
    verbose=True,
    tools=[SerperDevTool(n_results=3)],
    embedder_config={
        "provider": "ollama",
        "config": {
            "model": 'quentinz/bge-large-zh-v1.5:latest',
            "base_url": 'http://localhost:11434'
        }
    }
)

In [33]:
cv_agent,nlp_agent

(Agent(role=计算机视觉工程师, goal=查找并理解计算机视觉工程师的工作内容，熟悉的代码和工具, backstory=已经在计算机视觉领域工作5年，有丰富的计算机视觉经验),
 Agent(role=自然语言开发工程师, goal=查找并理解计算机视觉工程师的工作内容，熟悉的代码和工具, backstory=已经在自然语言开发领域工作10年，有丰富的自然语言开发经验))

Agent的初始化需要提供role、goal、backstory，最后还会绑定到特定任务上，这些概念有重叠，应该如何设计？以下列举官方的例子

|role|goal|backstory|
|---|---|---|
|研究分析师|查找并总结有关特定主题的信息|你是一位经验丰富的研究人员，注重细节|
|高级数据科学家|分析和解释复杂的数据集，提供可操作的见解|拥有超过10年的数据科学和机器学习经验，你擅长在复杂的数据集中发现模式|
|高级Python开发人员|编写和调试Python代码|拥有10年经验的专业Python开发人员|
|数据分析师|对大型数据集进行深入分析|擅长大数据分析和模式识别|
|客户服务代表|协助客户解决他们的问题|有丰富的客户支持经验，注重客户满意度|

可以看出以下内容：
- role：Agent的角色定位，相当于团队中的岗位
- goal：Agent的目标，相当于岗位职责
- backstory：Agent历史，该角色的历史经验、擅长内容